In [1]:
from urllib.request import urlopen as uReq
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
def download_img(img_url, title):
    my_url = f'https://image.tmdb.org/t/p/w342/{img_url}'
    response = requests.get(my_url, stream=True)
    file_size = int(response.headers.get("Content-Length", 0))
    
    with open('posters/' + img_url, "wb") as f:
        for data in response.iter_content(1024):
            f.write(data)
    return img_url


In [3]:
def get_data(id):
    try:
        movie_response = requests.get(f'{API_URL}/{id}{API_KEY}').json()
        title = movie_response['title']
        overview = movie_response['overview']
        poster_path = movie_response['poster_path']
        poster_path = download_img(poster_path)
        print(poster_path)

        video_response = requests.get(f'{API_URL}/{id}/videos{API_KEY}').json()
        video_key = None
        for vid in video_response['results']:
            if vid['type'] == 'Trailer':
                video_key = vid['key']
        
        if video_key is None:
            return pd.Series({'id': np.nan, 'title': np.nan, 'overview': np.nan, 'poster_path': np.nan, 'video_key': np.nan})

        return pd.Series({'id': id, 'title': title, 'overview': overview, 'poster_path': poster_path, 'video_key': video_key})
    except:
        return pd.Series({'id': np.nan, 'title': np.nan, 'overview': np.nan, 'poster_path': np.nan, 'video_key': np.nan})


In [4]:
links_df = pd.read_csv('ml-latest-small/links.csv')

In [5]:
API_KEY = '?api_key=9fe15a0c0fc071b3fab1b7cb6cf61dee'
API_URL = 'https://api.themoviedb.org/3/movie/'

In [6]:
res = links_df.loc[:10, 'tmdbId'].apply(get_data)

In [7]:
res

,id,title,overview,poster_path,video_key
0,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,tN1A2mVnrOM
1,8844.0,Jumanji,When siblings Judy and Peter discover an encha...,/vgpXmVaVyUL7GGiDeiK1mKEKzcX.jpg,eTjDsENDZ6s
2,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...,/1FSXpj5e8l4KH6nVFO5SPUeraOt.jpg,rEnOoWs3FuA
3,31357.0,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg,j9xml1CxgXI
4,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...,/rj4LBtwQ0uGrpBnCELr716Qo3mw.jpg,_Pv5qMpwrdU
5,949.0,Heat,"Obsessive master thief, Neil McCauley leads a ...",/obpPQskaVpSiC9RcJRB6iWDTCXS.jpg,14oNcFxiVaQ
6,11860.0,Sabrina,An ugly duckling having undergone a remarkable...,/z1oNjotUI7D06J4LWQFQzdIuPnf.jpg,iEqPw_QP28c
7,45325.0,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",/vIG8hWOa7DyLMRiurzKwVAnIYoU.jpg,-C-xXZyX2zU
8,9091.0,Sudden Death,When a man's daughter is suddenly taken during...,/1pylO6YX5XdOA6QCc5IRxrrffkg.jpg,SCOxEKkuWG4
9,710.0,GoldenEye,When a powerful satellite system falls into th...,/bFzjdy6ucvNlXmJwoSoYfufV6lP.jpg,mSBHbGduo5I
